### Request URL:
https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&page_limit=50&page_start=0
Request Method:
GET
Status Code:
200 OK

### Header 일부
:authority:
movie.douban.com
:method:
GET
:path:
/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&page_limit=50&page_start=0
:scheme:
https
Accept:
*/*
Accept-Encoding:
gzip, deflate, br, zstd
Accept-Language:
ko,en-US;q=0.9,en;q=0.8
Cookie:
bid=RGYdlPLQmzM; _pk_id.100001.4cf6=d434a2287554fbab.1721730526.; _pk_ses.100001.4cf6=1; ap_v=0,6.0; __utma=30149280.999071107.1721730527.1721730527.1721730527.1; __utmc=30149280; __utmz=30149280.1721730527.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); ll="108235"; __utma=223695111.1400437666.1721730539.1721730539.1721730539.1; __utmb=223695111.0.10.1721730539; __utmc=223695111;
Priority:
u=1, i
Referer:
https://movie.douban.com/
Sec-Ch-Ua:
"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"
Sec-Ch-Ua-Mobile:
?0
Sec-Ch-Ua-Platform:
"macOS"
Sec-Fetch-Dest:
empty
Sec-Fetch-Mode:
cors
Sec-Fetch-Site:
same-origin
User-Agent:
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36
X-Requested-With:
XMLHttpRequest

### Payload

type=movie&tag=%E7%83%AD%E9%97%A8&page_limit=50&page_start=0

### 응답 데이터 일부
{
    "subjects": [
        {
            "episodes_info": "",
            "rate": "8.0",
            "cover_x": 6000,
            "title": "走走停停",
            "url": "https:\/\/movie.douban.com\/subject\/35956190\/",
            "playable": true,
            "cover": "https://img1.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2907440658.jpg",
            "id": "35956190",
            "cover_y": 9000,
            "is_new": false
        },
        {
            "episodes_info": "",
            "rate": "9.1",
            "cover_x": 4056,
            "title": "泳者之心",
            "url": "https:\/\/movie.douban.com\/subject\/26656728\/",
            "playable": false,
            "cover": "https://img3.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2910815887.jpg",
            "id": "26656728",
            "cover_y": 6009,
            "is_new": false

해당 데이터를 목록으로 수집해서 sqlite db 로 저장하고 판다스 데이터프레임으로 저장하는 코드 작성

In [ ]:
import requests
import sqlite3
import pandas as pd

# Step 1: Fetch data from the given URL
url = "https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&page_limit=50&page_start=0"
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko,en-US;q=0.9,en;q=0.8",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

response = requests.get(url, headers=headers)
data = response.json()

# Step 2: Parse JSON response
movies = data['subjects']

# Step 3: Store data in SQLite database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies (
        id TEXT PRIMARY KEY,
        title TEXT,
        rate REAL,
        url TEXT,
        cover TEXT,
        cover_x INTEGER,
        cover_y INTEGER,
        playable BOOLEAN,
        is_new BOOLEAN
    )
''')

# Insert data into the table
for movie in movies:
    cursor.execute('''
        INSERT OR IGNORE INTO movies (id, title, rate, url, cover, cover_x, cover_y, playable, is_new)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (movie['id'], movie['title'], movie['rate'], movie['url'], movie['cover'], movie['cover_x'], movie['cover_y'], movie['playable'], movie['is_new']))

conn.commit()

# Step 4: Load data from SQLite database into pandas DataFrame
df = pd.read_sql_query('SELECT * FROM movies', conn)
conn.close()

# Display the DataFrame
df